In [3]:
########################################################################
## This is the main notebook to capture new images and test the model ##
########################################################################

import os
import cv2
import numpy as np
from datetime import datetime, timedelta

#--- SETTINGS --------------------------------------------------------------------------------------
# BGR COLORS
ROI_DEFAULT = (0, 255, 0)
ROI_SELECTION = (0, 200, 0)
ROI_CAPTURING = (0, 0, 255)
ROI_SAVING = (255, 255, 255)
ROI_PLAYING = (255, 0, 0)
ROI_BACKGROUND = (200, 200, 200)

# CAPTURE WINDOW
window_name = "Rock-Paper-Scissors"
capture_width = 1280
capture_height = 720

# ROI
default_roi_dim = 10, 10, 300, 300  # x, y, w, h
settings_path = "settings.txt"  # store some settings to file for next settion, None to disable

# CAPTURE SETTINGS
# image will be saved as "{folder}/{img_prefix}{img_next_id}.{img_extension}", example: captured/image_17.png.
folder = "captured"
img_prefix = "rock_"
img_extension = "png"
img_next_id = 0  # Starting id for image naming. If existing, the next highest id in the folder will be assigned
frames_batch = 5  # how many frames to capture in once session
capture_delay = 2000  # time in milliseconds to wait between captures

# PLAY SETTINGS
default_threshold = 20  # threshold to isolate hand from background in ROI
use_threshold = True  # can be toggled with 't' key
model = 'models/rps_v03_100epochs_20240208_1512.h5'
#---------------------------------------------------------------------------------------------------

# Access the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Error: Unable to open webcam")
    exit()

# Define capture window
cap.set(cv2.CAP_PROP_FRAME_WIDTH, capture_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, capture_height)
cv2.namedWindow(window_name)

# Load settings from file or use default values
try:
    # Load ROI dimensions from file or use default
    with open(settings_path, "r") as f:
        x, y, w, h, threshold_amount = map(int, f.readline().split())
        # reinit ROI if out of bounds
        if x < 0 or y < 0 or w < 0 or h < 0 or x + w > capture_width or y + h > capture_height:
            raise ValueError

except FileNotFoundError or ValueError:
    # use default values
    x, y, w, h = default_roi_dim
    threshold_amount = default_threshold

# ROI init settings
is_moving_roi = False
start_x, start_y = 0, 0
square_color = ROI_DEFAULT

# Capture init settings
is_capturing = False
time_counter = datetime.now()
image_counter = 0
os.makedirs(folder, exist_ok=True)

# Play init settings
is_playing = False
roi_background = None

# Mouse event callback function
def mouse_callback(event, mouseX, mouseY, flags, param):
    global x, y, w, h, is_moving_roi, start_x, start_y, square_color

    if event == cv2.EVENT_LBUTTONDOWN:
        # Check if mouse click is inside ROI
        if x <= mouseX <= x + w and y <= mouseY <= y + h:
            is_moving_roi = True
            start_x, start_y = mouseX - x, mouseY - y

    elif event == cv2.EVENT_MOUSEMOVE:
        if is_moving_roi:
            # Update ROI position while dragging
            x = max(0, min(mouseX - start_x, frame.shape[1] - w))
            y = max(0, min(mouseY - start_y, frame.shape[0] - h))

    elif event == cv2.EVENT_LBUTTONUP:
        is_moving_roi = False

cv2.setMouseCallback(window_name, mouse_callback)

while True:

    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check if the frame is read correctly
    if not ret:
        print("Error: Unable to capture frame")
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Get the ROI
    roi = frame[y:y+h, x:x+w]

    # Check for key press events
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c') and not is_playing:
        # toogle capturing mode
        is_capturing = False if is_capturing else True
    elif key == ord('p') and not is_capturing:
        # toogle playing mode
        is_playing = False if is_playing else True
        if is_playing:
            roi_background = roi.copy()
    elif key == 27:
        # Escape key, exit all modes
        is_capturing = False
        is_playing = False
        is_moving_roi = False
    elif key == ord('t'):
        # toogle activate use threshold
        use_threshold = not use_threshold
    elif key == ord('+') and threshold_amount < 255:
        # increase threshold
        threshold_amount += 1
    elif key == ord('-') and threshold_amount > 0:
        # decrease threshold
        threshold_amount -= 1
    elif key == ord('q'):
        # Quit the program
        break

    # Capturing Mode -------------------------------------------------------------------------------
    if is_capturing:
        if image_counter < frames_batch:
            if datetime.now() - time_counter > timedelta(milliseconds=capture_delay):
                # assign valid image id and name
                image_ids = [int(img.replace(img_prefix, "").replace(f".{img_extension}", "")) for img in os.listdir(folder)
                             if img.startswith(img_prefix) and img.endswith(f".{img_extension}")]
                if img_next_id in image_ids:
                    # assign next id
                    img_next_id = max(image_ids) + 1
                filepath = os.path.join(folder, f"{img_prefix}{img_next_id}.{img_extension}")
                # Save the ROI image
                cv2.imwrite(filepath, roi)
                image_counter += 1
                time_counter = datetime.now()
                square_color = ROI_SAVING
            else:
                square_color = ROI_CAPTURING
        elif image_counter == frames_batch:
            # reinit counter for next capture
            is_capturing = False
            image_counter = 0

    # Playing Mode ---------------------------------------------------------------------------------
    elif is_playing:
        square_color = ROI_PLAYING
        if use_threshold:
            # Compare ROI and background image, any identical pixels will be white
            diff = cv2.absdiff(roi, roi_background)
            mask = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(mask, threshold_amount, 255, cv2.THRESH_BINARY)
            mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
            new_bkg = np.ones_like(roi) * ROI_BACKGROUND
            frame[y:y+h, x:x+w] = np.where(mask==255, roi, new_bkg)

    # Moving ROI -----------------------------------------------------------------------------------
    elif is_moving_roi:
        square_color = ROI_SELECTION

    # Default Mode ---------------------------------------------------------------------------------
    else:
        square_color = ROI_DEFAULT

    # Draw square around the ROI
    cv2.rectangle(frame, (x, y), (x + w, y + h), square_color, 2)

    # Display the frame
    cv2.imshow(window_name, frame)

# Save settings
if settings_path:
    try:
        with open(settings_path, "w") as f:
            f.write(f"{x} {y} {w} {h} {threshold_amount}")
    except Exception as e:
        print(f"Unable to save settings to file: {e}")

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
